In [1]:
import os
import glob

import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import cv2

from keras.preprocessing import image
from keras.models import Model, Sequential
from keras.optimizers import Adam, RMSprop
from keras.callbacks import EarlyStopping
from keras.layers import Input, Reshape, Dense, MaxPooling2D, Activation, BatchNormalization, UpSampling2D, Flatten, Conv2D, Conv2DTranspose
from keras import backend as K

import random

np.random.seed(42)
os.environ["CUDA_VISIBLE_DEVICES"]="0" #model will be trained on GPU 0

Using TensorFlow backend.


In [3]:
def load_image(path):
    x_size = 28
    y_size = 28
    image_list = np.zeros((len(path), x_size, y_size, 1))
    for i, fig in enumerate(path):
        img = image.load_img(fig, color_mode='grayscale', target_size=(x_size, y_size))
        x = image.img_to_array(img).astype('float32')
        x = x / 255.0
        image_list[i] = x
    
    return image_list

In [4]:
train_img = glob.glob("../Data/Patches/Plaque/Aug/cal/*.png")
train_img += glob.glob("../Data/Patches/Plaque/Aug/fibrous/*.png")
train_img += random.sample(glob.glob("../Data/Patches/NoPlaque/*.png"), 3790)


test_img = glob.glob("dataset/Patches/Plaque/*.png")


random.shuffle(train_img)
random.shuffle(test_img)

x_train = load_image(train_img)
x_test = load_image(test_img) 

In [5]:
x_train.shape

(7580, 28, 28, 1)

In [28]:
batch_size = 128
epochs = 20
inChannel = 1
x, y = 28, 28
input_img = Input(shape = (x, y, inChannel))
print(input_img.shape)

(?, 28, 28, 1)


In [42]:

encoded = Dense(128, input_shape=(x,), activation='relu')(input_img)
encoded = Dense(64, activation='relu')(encoded)
encoded = Dense(32, activation='relu')(encoded)

decoded = Dense(64, activation='relu')(encoded)
decoded = Dense(128, activation='relu')(decoded)
decoded = Dense(784, activation='relu')(decoded)


In [43]:
"""
#flatten = Flatten()(input_img) 
encoded = Dense(4096, activation='relu')(input_img)
encoded = Dense(512, activation='relu')(encoded)
encoded = Dense(128, activation='relu')(encoded)
encoded = Dense(64, activation='relu')(encoded)

encoded = Dense(2, activation='relu')(encoded)

decoded = Dense(64, activation='relu')(encoded)
decoded = Dense(128, activation='relu')(decoded)
decoded = Dense(256, activation='relu')(decoded)
decoded = Dense(32, activation='relu')(decoded)
decoded = Dense(1, activation='relu')(decoded)
print(decoded.shape)
"""

"\n#flatten = Flatten()(input_img) \nencoded = Dense(4096, activation='relu')(input_img)\nencoded = Dense(512, activation='relu')(encoded)\nencoded = Dense(128, activation='relu')(encoded)\nencoded = Dense(64, activation='relu')(encoded)\n\nencoded = Dense(2, activation='relu')(encoded)\n\ndecoded = Dense(64, activation='relu')(encoded)\ndecoded = Dense(128, activation='relu')(decoded)\ndecoded = Dense(256, activation='relu')(decoded)\ndecoded = Dense(32, activation='relu')(decoded)\ndecoded = Dense(1, activation='relu')(decoded)\nprint(decoded.shape)\n"

In [44]:
"""
flatten = Flatten()(input_img) 
encoded = Dense(2048, activation = 'relu')(flatten)
encoded = Dense(2048, activation = 'relu')(encoded)
encoded = Dense(8, activation = 'relu')(encoded)
encoded = Dense(2)(encoded)

print(encoded.shape) 
decoded = Dense(8, activation = 'relu')(encoded)
decoded = Dense(2048, activation = 'relu')(decoded)
decoded = Dense(2048, activation = 'relu')(decoded)
decoded = Dense(8*8*8, activation = 'relu')(decoded)
decoded = Reshape((8,8,8))(decoded)
print("decoded: ", decoded.shape)
"""

'\nflatten = Flatten()(input_img) \nencoded = Dense(2048, activation = \'relu\')(flatten)\nencoded = Dense(2048, activation = \'relu\')(encoded)\nencoded = Dense(8, activation = \'relu\')(encoded)\nencoded = Dense(2)(encoded)\n\nprint(encoded.shape) \ndecoded = Dense(8, activation = \'relu\')(encoded)\ndecoded = Dense(2048, activation = \'relu\')(decoded)\ndecoded = Dense(2048, activation = \'relu\')(decoded)\ndecoded = Dense(8*8*8, activation = \'relu\')(decoded)\ndecoded = Reshape((8,8,8))(decoded)\nprint("decoded: ", decoded.shape)\n'

In [45]:
"""
conv1 = Conv2D(16, (3, 3), activation='relu', padding='same')(input_img)
pool1 = MaxPooling2D((2, 2), padding='same')(conv1)
conv2 = Conv2D(8, (3, 3), activation='relu', padding='same')(pool1)
pool2 = MaxPooling2D((2, 2), padding='same')(conv2)
conv3 = Conv2D(8, (3, 3), activation='relu', padding='same')(pool2)
flatten1 = Flatten()(conv3)
print(flatten1.shape)
#reshape = Reshape((8,8,8))(flatten1)
encoded = Dense(8, activation='relu')(flatten1)
encoded2 = Dense(2)(encoded)
#encoded = MaxPooling2D((2, 2), padding='same')(flatten1)
print (encoded.shape)


decoder = Dense()
conv4 = Conv2DTranspose(8, (3, 3), activation='relu', padding='same')(reshape)
up1 = UpSampling2D((2, 2))(conv4)
conv5 = Conv2D(8, (3, 3), activation='relu', padding='same')(up1)
up2 = UpSampling2D((2, 2))(conv5)
conv6 = Conv2D(16, (3, 3), activation='relu', padding = 'same')(up2)
up3 = UpSampling2D((2, 2))(conv6)
decoded = Conv2D(1, (3, 3), activation='relu', padding='same')(up3)
encoded = Dense(units=128, activation='relu')(input_img)
encoded = Dense(units=64, activation='relu')(encoded)
encoded = Dense(units=32, activation='relu')(encoded)
decoded = Dense(units=64, activation='relu')(encoded)
decoded = Dense(units=128, activation='relu')(decoded)
decoded = Dense(units=784, activation='sigmoid')(decoded)
"""

"\nconv1 = Conv2D(16, (3, 3), activation='relu', padding='same')(input_img)\npool1 = MaxPooling2D((2, 2), padding='same')(conv1)\nconv2 = Conv2D(8, (3, 3), activation='relu', padding='same')(pool1)\npool2 = MaxPooling2D((2, 2), padding='same')(conv2)\nconv3 = Conv2D(8, (3, 3), activation='relu', padding='same')(pool2)\nflatten1 = Flatten()(conv3)\nprint(flatten1.shape)\n#reshape = Reshape((8,8,8))(flatten1)\nencoded = Dense(8, activation='relu')(flatten1)\nencoded2 = Dense(2)(encoded)\n#encoded = MaxPooling2D((2, 2), padding='same')(flatten1)\nprint (encoded.shape)\n\n\ndecoder = Dense()\nconv4 = Conv2DTranspose(8, (3, 3), activation='relu', padding='same')(reshape)\nup1 = UpSampling2D((2, 2))(conv4)\nconv5 = Conv2D(8, (3, 3), activation='relu', padding='same')(up1)\nup2 = UpSampling2D((2, 2))(conv5)\nconv6 = Conv2D(16, (3, 3), activation='relu', padding = 'same')(up2)\nup3 = UpSampling2D((2, 2))(conv6)\ndecoded = Conv2D(1, (3, 3), activation='relu', padding='same')(up3)\nencoded = Den

In [46]:
autoencoder = Model(input_img, decoded)
autoencoder.summary()
encoder = Model(input_img, encoded)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 28, 28, 1)         0         
_________________________________________________________________
dense_25 (Dense)             (None, 28, 28, 128)       256       
_________________________________________________________________
dense_26 (Dense)             (None, 28, 28, 64)        8256      
_________________________________________________________________
dense_27 (Dense)             (None, 28, 28, 32)        2080      
_________________________________________________________________
dense_28 (Dense)             (None, 28, 28, 64)        2112      
_________________________________________________________________
dense_29 (Dense)             (None, 28, 28, 128)       8320      
_________________________________________________________________
dense_30 (Dense)             (None, 28, 28, 784)       101136    
Total para

In [47]:
autoencoder.compile(optimizer='adam', loss='mse')
autoencoder.fit(x_train, x_train,
                epochs=epochs,
                batch_size=batch_size,
                validation_data=(x_test, x_test))

ValueError: Error when checking target: expected dense_30 to have shape (28, 28, 784) but got array with shape (28, 28, 1)

In [ ]:
encoded2.shape

In [48]:
y_test = []

for img in test_img:
    if "Calcific" in img:
        y_test.append(0)
    elif "Fibrous" in img:
        y_test.append(1)

In [ ]:
label_conv = {
    0: 'Calcific',
    1: 'Fibrous'
}

In [ ]:
num_images = 10
random_test_images = np.random.randint(x_test.shape[0], size=num_images)

encoded_imgs = encoder.predict(x_test)
decoded_imgs = autoencoder.predict(x_test)

print(encoded_imgs[1])

plt.figure(figsize=(18, 4))

for i, image_idx in enumerate(random_test_images):
    # plot original image
    ax = plt.subplot(3, num_images, i + 1)
    plt.title(label_conv[y_test[i]])
    plt.imshow(x_test[image_idx].reshape(64, 64))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    '''
    # plot encoded image
    print(encoded_imgs[0].shape)
    ax = plt.subplot(3, num_images, num_images + i + 1)
    plt.imshow(encoded_imgs[image_idx].reshape(2,2))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    '''
    # plot reconstructed image
    ax = plt.subplot(3, num_images, 2*num_images + i + 1)
    plt.imshow(decoded_imgs[image_idx].reshape(64, 64))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()

In [ ]:
encoded_imgs[0].shape

In [ ]:
encoded_imgs[0]

In [ ]:
from __future__ import print_function
import time

import numpy as np
import pandas as pd

from sklearn.datasets import fetch_mldata
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

%matplotlib inline
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

import seaborn as sns

In [ ]:
tsne = TSNE(n_components=3, verbose=1, perplexity=40, n_iter=300)
embedding = tsne.fit_transform(encoded_imgs)

In [ ]:
len(y_test)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.scatter(encoded_imgs[:, 0], encoded_imgs[:, 1])

In [ ]:

plt.figure(figsize=(10, 10))
plt.scatter(encoded_imgs[:, 0], encoded_imgs[:, 1], c=y_test, cmap='brg')
plt.colorbar()
plt.show()

In [ ]:
import matplotlib.cm as cm

colors = cm.rainbow(np.linspace(0, 1, len(y_test)))
plt.scatter(encoded_imgs[:, 0], encoded_imgs[:, 1], c=)
plt.colorbar()
plt.show()

In [ ]:
embedding.shape

In [ ]:
embedding = pd.DataFrame(embedding, columns=['x','y', 'z'])
embedding['hue'] = y_test

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

cos_sim = cosine_similarity(encoded_imgs[0].reshape(1,-1), encoded_imgs[1].reshape(1,-1))

In [ ]:
cos_sim